In [6]:
root = '/home/lioruzan/word_spotting/comp/Segmentation-based_Dataset/Cand3'
data_dir = '/home/lioruzan/pixel-cnn/data'

In [7]:
import glob
flist = [name for name in glob.glob(root+'/*/*.jpg')]
    

In [8]:
import scipy.misc as sm
imgs = [sm.imread(file,mode='RGB') for file in flist]

In [9]:
resized_imgs = [sm.imresize(im,(32,32)) for im in imgs]

In [10]:
import numpy as np
resized_imgs = [im[np.newaxis,:,:,:] for im in resized_imgs]
images = np.vstack(resized_imgs)

In [11]:
p = np.random.permutation(images.shape[0])
images = images[p]
test = images[:200]
train = images[200:]

In [20]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import feature
import scipy.misc as sm
import cv2

def get_edges(imname):
    root = '/home/lioruzan/pixel-cnn/data'
    img = sm.imread(os.path.join(root,imname))
    kernel = np.ones((5,5),np.uint8)
    im = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    # Compute the Canny filter
    edges = feature.canny(im, sigma=3)
    #print(edges.shape)
    return edges
    

In [21]:
t=384
masklist = []
for imgname, curset in [('P976-Fg001-R-C01-R02-D04082013-T145409-LR924 _012_mask.jpg', train),
                        ('P976-Fg004-R-C01-R02-D04082013-T143613-LR924 _012_mask.jpg', test)]:
    edges = get_edges(imname)
    x,y = np.nonzero(edges)
    masks = np.zeros_like(curset)
    for c in range(curset.shape[0]):
        cc = np.random.randint(len(x))
        i, j = x[cc], y[cc]
        while i<t/2 or i>=edges.shape[0]-t/2 or j<t/2 or j>=edges.shape[1]-t/2:
            cc = np.random.randint(len(x))
            i,j = x[cc], y[cc]
        patch = im[i-t/2:i+t/2,j-t/2:j+t/2]
        patch = sm.imresize(patch, (32,32))[:,:,np.newaxis]
        masks[c] = patch
        #sm.imshow(patch)
    masklist.append(masks)
    
train_masks = masklist[0]
test_masks  = masklist[1]

/home/lioruzan/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [23]:
# sm.imshow(test_masks[3])

In [24]:
import pickle as pkl
import os
with open(os.path.join(data_dir,'letters_train.pkl'),'wb') as f:
    pkl.dump({'data':train,'labels':None,'masks':train_masks},f)
with open(os.path.join(data_dir,'letters_test.pkl'),'wb') as f:
    pkl.dump({'data':test,'labels':None,'masks':test_masks},f)